In [3]:
import PyPDF2
import tensorflow as tf
import numpy as np

In [16]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = len(reader.pages)
        for page_num in range(num_pages):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

In [17]:
# Function to generate text based on user input prompt
def generate_text_from_prompt(prompt, next_words, model, max_sequence_len):
    generated_text = prompt.lower()  # Start the generated text with the user prompt
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([prompt])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        prompt += " " + output_word
        generated_text += " " + output_word
    return generated_text

In [18]:
# Extract text from PDF
pdf_path = r'/content/ilovepdf_merged_removed.pdf'
extracted_text = extract_text_from_pdf(pdf_path)

In [19]:
# Tokenize the text and create sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts([extracted_text])
total_words = len(tokenizer.word_index) + 1

In [24]:
input_sequences = []
for line in extracted_text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(tf.keras.preprocessing.sequence.pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Create predictors and labels
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

# Define a more complex LSTM model
models = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(total_words, 100, input_length=max_sequence_len-1),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1024, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(total_words, activation='softmax')
])

# Custom learning rate scheduler
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

models.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
print(models.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 64, 100)           439800    
                                                                 
 bidirectional_4 (Bidirecti  (None, 64, 2048)          9216000   
 onal)                                                           
                                                                 
 bidirectional_5 (Bidirecti  (None, 64, 1024)          10489856  
 onal)                                                           
                                                                 
 bidirectional_6 (Bidirecti  (None, 64, 512)           2623488   
 onal)                                                           
                                                                 
 bidirectional_7 (Bidirecti  (None, 256)               656384    
 onal)                                                

In [ ]:
# Train the model with adjusted epochs
history = models.fit(xs, ys, epochs=100, verbose=1)

Epoch 1/100
1521/1521 [==============================] - 198s 121ms/step - loss: 6.7436 - accuracy: 0.0448
Epoch 2/100
1521/1521 [==============================] - 178s 117ms/step - loss: 6.5976 - accuracy: 0.0452
Epoch 3/100
1521/1521 [==============================] - 178s 117ms/step - loss: 6.5855 - accuracy: 0.0452
Epoch 4/100
1521/1521 [==============================] - 177s 116ms/step - loss: 6.5789 - accuracy: 0.0452
Epoch 5/100
1521/1521 [==============================] - 177s 116ms/step - loss: 6.5759 - accuracy: 0.0452
Epoch 6/100
1521/1521 [==============================] - 177s 116ms/step - loss: 6.5745 - accuracy: 0.0452
Epoch 7/100
1521/1521 [==============================] - 177s 116ms/step - loss: 6.5720 - accuracy: 0.0452
Epoch 8/100
1521/1521 [==============================] - 177s 116ms/step - loss: 6.5687 - accuracy: 0.0452
Epoch 9/100
1521/1521 [==============================] - 177s 116ms/step - loss: 6.5673 - accuracy: 0.0452
Epoch 10/100
1154/1521 [=============

In [21]:
# Get user prompt
user_prompt = input("Enter your prompt: ")

# Function to generate text
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Generate text based on user prompt
next_words = 50
generated_text = generate_text_from_prompt(user_prompt, next_words, model, max_sequence_len)
print(generated_text)

Epoch 1/300
1521/1521 [==============================] - 197s 120ms/step - loss: 6.7442 - accuracy: 0.0449
Epoch 2/300
1521/1521 [==============================] - 178s 117ms/step - loss: 6.5950 - accuracy: 0.0452
Epoch 3/300
1521/1521 [==============================] - 177s 116ms/step - loss: 6.5827 - accuracy: 0.0452
Epoch 4/300
1521/1521 [==============================] - 178s 117ms/step - loss: 6.5792 - accuracy: 0.0452
Epoch 5/300
1521/1521 [==============================] - 178s 117ms/step - loss: 6.5754 - accuracy: 0.0452
Epoch 6/300
1521/1521 [==============================] - 177s 116ms/step - loss: 6.5741 - accuracy: 0.0452
Epoch 7/300
1521/1521 [==============================] - 177s 116ms/step - loss: 6.5714 - accuracy: 0.0452
Epoch 8/300
1521/1521 [==============================] - 177s 116ms/step - loss: 6.5680 - accuracy: 0.0452
Epoch 9/300
1521/1521 [==============================] - 177s 117ms/step - loss: 6.5674 - accuracy: 0.0452
Epoch 10/300
1521/1521 [=============

KeyboardInterrupt: 

In [23]:
user_prompt = input("Enter your prompt: ")

# Function to generate text
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = tf.keras.preprocessing.sequence.pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Generate text based on user prompt
next_words = 50
generated_text = generate_text_from_prompt(user_prompt, next_words, model, max_sequence_len)
print(generated_text)

Enter your prompt: tailwind
1/1 [==============================] - 0s 30ms/step
tailwind the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
